
# xgBoost
模型
$$
\hat{y}=\sum_{m=1}^M f_m(x)
$$
损失函数记为
$$
\mathcal{L}(y,\hat{y})=\sum_{i=1}^Nl(y_i,\hat{y}_i)+\sum_{m=1}^M\Omega(f_m)
$$
（要求二阶可导）。
$$
\Omega(f)=\gamma T+\frac{1}{2}\lambda||w||^2
$$
为正则化项，T为叶节点树，w为叶节点权重。

## 叶节点最优权重
第t次迭代
$\hat{y}_i^{(t)}=\hat{y}_i^{(t-1)}+f_t(x_i),i=1,2,\cdots,N$.
优化目标为
$$
argmin_{f_t}\sum_{i=1}^N[l(y_i,\hat{y}_i^{(t-1)}+f_t(x_i))]+\Omega(f_t)\\
$$
$l$在$\hat{y}_i^{(t-1)}$处Taylor展开：
$$
l(y_i,\hat{y}_i^{(t-1)}+f_t(x_i))=l(y_i,\hat{y}_i^{(t-1)})+g_if_t(x_i) + \frac{1}{2}h_i{f_t(x_i)}^2
$$
其中
$$
g_i = \frac{\partial l(y_i,\hat{y}_i)}{\partial \hat{y}_i}|_{\hat{y}_i=\hat{y}_i^{(t-1)}}
$$
$$
h_i = \frac{\partial^2 l(y_i,\hat{y}_i)}{\partial \hat{y}_i^2}|_{\hat{y}_i=\hat{y}_i^{(t-1)}}
$$
故优化目标化为
$$
argmin_{f_t}\sum_{i=1}^N[g_if_t(x_i)+\frac{1}{2}h_if_t(x_i)^2]+\gamma T + \frac{1}{2}\lambda||w||^2\\
=argmin_{f_t}\sum_{j=1}^T[(\sum_{i \in I_j}g_i)w_j+\frac{1}{2}(\sum_{i\in I_j}h_i+\lambda)w_j^2]
$$
对$w_j$求导并置零，有
$$
w^*_j=-\frac{\sum_{i\in I_j}g_i}{(\sum_{i\in I_j}h_i+\lambda)}
$$
对应的最优损失函数为
$$
\mathcal{L}^{(t)}=-\frac{1}{2}\sum_{j=1}^T\frac{(\sum_{i\in I_j}g_i)^2}{\sum_{i\in I_j}h_i+\lambda} + \gamma T
$$

## 树的生成
遍历所有特征的所有特征划分点，根据划分后的增益来决定是否划分。

设划分后左右节点集合为$I_L,I_R$.

$$
Gain = \mathcal{L}_I - (\mathcal{L}_{I_L}+\mathcal{L}_{I_R})\\
=\frac{1}{2}[\frac{(\sum_{i\in I_L}g_i)^2}{\sum_{i\in I_L}h_i+\lambda}+\frac{(\sum_{i\in I_R}g_i)^2}{\sum_{i\in I_R}h_i+\lambda}-\frac{(\sum_{i\in I}g_i)^2}{\sum_{i\in I}h_i+\lambda}] - \gamma T
$$
为了限制生长过深，可以给定一个阀值，但Gain大于阀值时才分裂。
同时还可以设置最大深度，样本权重小于阀值时停止生长等来防止过拟合。